In [1]:
def calculer_nre_par_rb(N_sc_rb=12, N_symb_sh=14, N_overhead=0):
    """
    Calculer le nombre de REs (Resource Elements) par RB (Resource Block).

    Paramètres :
    - N_sc_rb : Nombre de sous-porteuses par RB (Resource Block), généralement 12.
    - N_symb_sh : Nombre de symboles OFDM dans un slot, généralement 14.
    - N_overhead : Éléments de ressource réservés pour le DMRS et les couches supérieures.
    
    Retourne :
    - Nombre de REs par RB.
    """
    N_re_rb = N_sc_rb * N_symb_sh - N_overhead
    return N_re_rb

def calculer_volume_information(N_rb, N_re_rb, efficacite):
    """
    Calculer le volume d'information transmissible pour un certain nombre de RBs.

    Paramètres :
    - N_rb : Nombre de RBs disponibles.
    - N_re_rb : Nombre de REs par RB.
    - efficacite : Efficacité de la transmission, donnée par la table de CQI.
    
    Retourne :
    - Volume d'information transmissible (en bits).
    """
    volume_information = N_rb * N_re_rb * efficacite
    return volume_information

def calculer_rbs_totales(antenna):
    """
    Calculer le nombre de Resource Blocks (RBs) disponibles sans dépasser le maximum autorisé.

    Paramètres :
    - largeur_bande_mhz : Largeur de bande totale du canal en Megahertz.
    - espacement_sous_porteuses_khz : Espacement entre les sous-porteuses en Kilohertz.

    Retourne :
    - Nombre total de RBs disponibles sans dépasser la configuration maximale.
    """
    # Tableau de configuration maximale de bande passante (NRB) par SCS et bande passante
    max_nrb_config = {
        15: {5: 25, 10: 52, 15: 79, 20: 106, 25: 133, 30: 160, 35: 188, 40: 216, 45: 242, 50: 270},
        30: {5: 11, 10: 24, 15: 38, 20: 51, 25: 65, 30: 78, 35: 92, 40: 106, 45: 119, 50: 132, 60: 162, 70: 189, 80: 217, 90: 245, 100: 273},
        60: {10: 11, 15: 18, 20: 24, 25: 31, 30: 38, 35: 44, 40: 51, 45: 58, 50: 65, 60: 79, 70: 93, 80: 107, 90: 121, 100: 135}
    }

    # Conversion des paramètres d'entrée en conformité avec le tableau (MHz en Hz et kHz en Hz)
    largeur_bande_hz = antenna.bandwidth * 1e6
    espacement_sous_porteuses_hz = antenna.sub_carrier_spacing * 1e3

    # Calcul du nombre de RBs en fonction de la largeur de bande et de l'espacement des sous-porteuses
    nombre_sous_porteuses_par_rb = 12
    largeur_bande_rb_hz = espacement_sous_porteuses_hz * nombre_sous_porteuses_par_rb
    nombre_rbs = int(largeur_bande_hz / largeur_bande_rb_hz)
    
    # Vérification pour ne pas dépasser le maximum autorisé selon la configuration
    nrb_max_autorise = max_nrb_config.get(antenna.sub_carrier_spacing, {}).get(antenna.bandwidth, None)
    if nrb_max_autorise is not None:
        return min(nombre_rbs, nrb_max_autorise)
    else:
        return nombre_rbs  # Ou retourner None ou lever une exception si la configuration n'est pas valide


def calculer_rbs_disponibles(antenna, ue): # !! Puisque c'est une liste, comment extraire la Tx qui nous intéresse??
    slot_duration_ms = {15: 1, 30: 0.5, 60: 0.25}[antenna.sub_carrier_spacing]  # Durée d'un slot en ms en fonction de SCS
    bits_per_rb = ...  # À déterminer en fonction de l'efficacité du codage (CQI)

    # Calculer la durée de la transmission pour l'UE
    duration_ms = ue.end_tx - ue.start_tx 

    # Calculer le nombre total de slots utilisés
    total_slots = duration_ms / slot_duration_ms

    # Calculer le nombre de RBs nécessaires pour les données transmises
    total_rbs_for_data = ue.Tx_bits / bits_per_rb

    # Estimer le nombre total de RBs utilisés, en prenant en compte que plusieurs RBs peuvent être utilisés par slot
    total_rbs_used = total_slots * total_rbs_for_data

    total_rbs_disponibles = calculer_rbs_totales(antenna) - total_rbs_used

    return total_rbs_disponibles


In [2]:

# Exemple d'utilisation
N_overhead = 2  # Par exemple, dépendant de la configuration
efficacite = 4  # Hypothétique, doit être déterminé par la table de CQI
N_rb = 100  # Nombre de RBs disponibles, hypothétique

N_re_rb = calculer_nre_par_rb(N_overhead=N_overhead)
volume_info = calculer_volume_information(N_rb, N_re_rb, efficacite)

print(f"Nombre de REs par RB: {N_re_rb}")
print(f"Volume d'information transmissible: {volume_info} bits")

# Exemple d'utilisation
largeur_bande_hz = 100e6  # 100 MHz, par exemple
espacement_sous_porteuses_hz = 15e3  # 15 kHz, par exemple

# Exemple qui ne dépasse pas la limite maximale autorisée
largeur_bande_mhz_non_depassant = 20  # 20 MHz
espacement_sous_porteuses_khz = 15  # 15 kHz
nombre_rbs_non_depassant = calculer_rbs_totales(antenna)
print(f"Nombre de RBs disponibles: {nombre_rbs_non_depassant}")
# Exemple qui dépasse la limite maximale autorisée
# Choisissons une largeur de bande qui n'est pas présente dans le tableau pour cet espacement
# comme 7 MHz pour un espacement de 15 kHz qui n'est pas une configuration standard.
largeur_bande_mhz_depassant = 7  # 7 MHz (configuration non standard)
nombre_rbs_depassant = calculer_rbs_totales(antenna)
print(f"Nombre de RBs disponibles: {nombre_rbs_depassant}")

Nombre de REs par RB: 166
Volume d'information transmissible: 66400 bits


TypeError: calculer_nombre_rbs() takes 1 positional argument but 2 were given